In [7]:
import requests
import pandas as pd
import cv2
import urllib.request
import numpy as np
from matplotlib import pyplot as plt
#from keras.preprocessing import image

%matplotlib inline


%store -r WORKDIR

#if 'WORKDIR' not in dir():
WORKDIR = 'C:/Users/cammy/OneDrive/MIT IA/git/projeto_dogs/dogs_brand'



baseurl = 'https://dog.ceo/api/'

listurl = baseurl + 'breeds/list/all'
randomappend = '/images/random/'
allimages = '/images'

data_raw_file = WORKDIR + '/Data/Raw/dogs.parquet'

maxBreeds = 3
maxImages = 0

img_h, img_w = 64, 64 # Altura e largura das imagens

#breeds = []

In [8]:
WORKDIR

'C:/Users/cammy/OneDrive/MIT IA/git/projeto_dogs/dogs_brand'

# Busca Lista de Raças

In [9]:
obj = requests.get(listurl).json()
allbreeds = pd.DataFrame.from_dict(obj)
allbreeds.head()

,message,status
affenpinscher,[],success
african,[],success
airedale,[],success
akita,[],success
appenzeller,[],success


In [10]:
breeds = allbreeds.index
if maxBreeds > 0:
    breeds = allbreeds [0:maxBreeds].index
    

# Ler imagens

In [11]:
def ler_imagem(url):

# For remote, use urllib, as per "http://stackoverflow.com/questions/21061814"
    req = urllib.request.urlopen(url)

    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
# 0 = monocromatic
    img = cv2.imdecode(arr,0)
    imres = cv2.resize(img, (img_h, img_w), interpolation=cv2.INTER_CUBIC)


# Plot
#    plt.figure()  
#    plt.imshow(imres)
    return imres

In [ ]:
imageurls = []
dogimages = []
rotulos = []
#dataset = np.ndarray((len(breeds) * maxImages, img_h, img_w, 3), dtype=np.uint8)
i = 0

for breed in breeds:
    filesurl = ''
    if maxImages > 0:
        filesurl = baseurl + 'breed/' + breed + randomappend + str(maxImages)
    else:
        filesurl = baseurl + 'breed/' + breed + allimages
    print(filesurl)
    imageurl = requests.get(filesurl).json()
    print(imageurl)
    breedimages = pd.DataFrame.from_dict(imageurl['message'])
    imageurls.append(breedimages)
    for images in breedimages.values:
        img = ler_imagem(images[0])
#        dataset[i] = img
        rotulos.append(breed)
        dogimages.append(img.flatten())
        i = i+1

https://dog.ceo/api/breed/affenpinscher/images
{'message': ['https://images.dog.ceo/breeds/affenpinscher/n02110627_10147.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10185.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10225.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10437.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10439.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10447.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10680.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10787.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10848.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10859.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10986.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11211.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11262.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11263.jpg', 'h

In [ ]:
rotulos

In [ ]:
#dataset.shape

In [ ]:
dogimages

In [ ]:
# view an image (e.g. 20) and print its corresponding label
#img_index = 10
#plt.imshow(dataset[img_index,:,:,:])
#plt.show()
#print(rotulos[img_index])

In [ ]:
df = pd.DataFrame(dogimages)
df['breed'] = rotulos

In [ ]:
df

In [ ]:
df.columns = df.columns.astype(str)
df.to_parquet(data_raw_file)